# NLP基礎 (2): 言語モデリング (Language Modeling) と評価指標

このノートブックでは、自然言語処理 (NLP) の中核的なタスクの一つである**言語モデリング (Language Modeling)** について学びます。
特に、古典的かつ基本的な手法である **n-gram言語モデル** の概念、その学習方法（最尤推定とスムージING）、そして言語モデルの性能を測るための主要な評価指標である**パープレキシティ (Perplexity)** について解説します。
NumPyを使って、これらの概念をスクラッチで実装し、その動作を理解します。

**参考論文:**
*   Chen, S. F., & Goodman, J. (1996). An empirical study of smoothing techniques for language modeling. (参照した論文。様々なスムージング手法を比較)
*   Jelinek, F., & Mercer, R. L. (1980). Interpolated estimation of Markov source parameters from sparse data. (線形補間スムージングの基礎)
*   Katz, S. M. (1987). Estimation of probabilities from sparse data for the language model component of a speech recognizer. *IEEE transactions on acoustics, speech, and signal processing*, 35(3), 400-401. (Katz Backoff)

**このノートブックで学ぶこと:**
1.  言語モデルの目的と基本的な考え方。
2.  n-gram言語モデルの定義、確率計算、最尤推定。
3.  データスパースネス問題と、その解決策としてのスムージング手法（特にラプラススムージングと線形補間の概念）。
4.  言語モデルの評価指標である負対数尤度 (NLL) とパープレキシティ (PPL) の定義と計算。
5.  NumPyによるn-gramモデルの構築とPPL計算の実装。

**前提知識:**
*   テキスト前処理、Bag-of-Words、TF-IDFの基本的な理解（NLP基礎(1)）。
*   確率の基本的な概念（条件付き確率など）。
*   Pythonの基本的なデータ構造とNumPyの操作。

## 1. 必要なライブラリのインポート

In [8]:
import numpy as np
from collections import Counter, defaultdict
import math

## 2. 言語モデルとは？

**言語モデル (Language Model, LM)** とは、単語のシーケンス（文やフレーズなど）がどれだけ「自然」か、つまりそのシーケンスが出現する**確率**を与えるモデルです。

$P(W) = P(w_1, w_2, \dots, w_m)$

ここで、$W = (w_1, w_2, \dots, w_m)$ は単語のシーケンスです。

言語モデルは、NLPの多くの応用で中心的な役割を果たします。
*   **機械翻訳:** 生成される翻訳文が自然な文かどうかを評価する。
*   **音声認識:** 音響モデルからの複数の候補のうち、より自然な単語列を選択する。
*   **スペル訂正・かな漢字変換:** 最も確率の高い正しい単語列を提案する。
*   **テキスト生成:** 次に来る単語を予測し、自然な文章を生成する。

条件付き確率の連鎖律を用いると、シーケンスの同時確率は以下のように分解できます。
$P(w_1, w_2, \dots, w_m) = P(w_1) \times P(w_2 | w_1) \times P(w_3 | w_1, w_2) \times \dots \times P(w_m | w_1, \dots, w_{m-1})$
$P(W) = \prod_{i=1}^{m} P(w_i | w_1, \dots, w_{i-1})$

この式は、各単語の出現確率を、それ以前に出現した全ての単語の履歴に基づいて計算することを示しています。

## 3. n-gram言語モデル

上記の条件付き確率 $P(w_i | w_1, \dots, w_{i-1})$ を正確に推定するには、非常に長い履歴を考慮する必要があり、データ量の観点から困難です（データスパースネス問題）。

**n-gram言語モデル**は、この問題を解決するために、**マルコフ仮定**を導入します。マルコフ仮定とは、「現在の単語 $w_i$ の出現確率は、直前の $n-1$ 個の単語の履歴にのみ依存する」という仮定です。

$P(w_i | w_1, \dots, w_{i-1}) \approx P(w_i | w_{i-n+1}, \dots, w_{i-1})$

*   **Unigram (1-gram) モデル ($n=1$):** 各単語は独立に出現すると仮定。
    $P(w_i | w_1, \dots, w_{i-1}) \approx P(w_i)$
*   **Bigram (2-gram) モデル ($n=2$):** 現在の単語は直前の1単語のみに依存。
    $P(w_i | w_1, \dots, w_{i-1}) \approx P(w_i | w_{i-1})$
*   **Trigram (3-gram) モデル ($n=3$):** 現在の単語は直前の2単語のみに依存。
    $P(w_i | w_1, \dots, w_{i-1}) \approx P(w_i | w_{i-2}, w_{i-1})$

一般に、$n$ が大きいほどより長い文脈を考慮できますが、データスパースネス問題が深刻になります。

### 3.1 n-gram確率の最尤推定 (Maximum Likelihood Estimation - MLE)

n-gramの条件付き確率は、訓練コーパスにおける出現頻度から最尤推定で計算できます。
例えば、Bigram確率 $P(w_i | w_{i-1})$ は、

$P_{ML}(w_i | w_{i-1}) = \frac{\text{Count}(w_{i-1}, w_i)}{\text{Count}(w_{i-1})}$

Trigram確率 $P(w_i | w_{i-2}, w_{i-1})$ は、

$P_{ML}(w_i | w_{i-2}, w_{i-1}) = \frac{\text{Count}(w_{i-2}, w_{i-1}, w_i)}{\text{Count}(w_{i-2}, w_{i-1})}$

ここで、$\text{Count}(\cdot)$ はコーパス中でのn-gram（またはそのプレフィックス）の出現回数です。

### 3.2 データスパースネスとゼロ確率問題

最尤推定には大きな問題があります。訓練コーパスが有限であるため、実際にはあり得るn-gramでも、たまたま訓練コーパスに出現しなかったものの確率は0になってしまいます（**ゼロ確率問題**）。
例えば、"thank you very much" という自然なフレーズがあっても、訓練コーパスに "thank you very" というtrigramが出現しなかった場合、$P(\text{much} | \text{thank you very}) = 0$ となり、このフレーズ全体の確率も0になってしまいます。これは明らかに不適切です。
この問題を解決するために、**スムージング (Smoothing)** というテクニックが使われます。

## 4. スムージング手法

スムージングは、観測されなかったn-gramにもゼロでない小さな確率を割り当て、観測されたn-gramの確率を少し割り引くことで、確率の総和が1になるように調整する手法です。

### 4.1 ラプラススムージング (Add-one Smoothing)

最も単純なスムージング手法で、全てのn-gramの出現回数に1を加えてから確率を計算します。
例えば、Bigramの場合:

$P_{Laplace}(w_i | w_{i-1}) = \frac{\text{Count}(w_{i-1}, w_i) + 1}{\text{Count}(w_{i-1}) + V}$

ここで、$V$ は語彙サイズです。分母に $V$ を加えるのは、全ての可能な次の単語についてカウントが1増えるため、確率の総和を1に保つためです。

*   **利点:** 実装が非常に簡単で、ゼロ確率を防げます。
*   **欠点:** 訓練コーパスに出現しなかったn-gramに過大な確率質量を割り当ててしまう傾向があります。特に大規模な語彙の場合、この影響が大きくなります。

一般的には、より洗練されたAdd-k Smoothing（1の代わりに小さな定数 $k$ を加える）も使われます。

In [9]:
# 簡単なコーパスでテスト
test_corpus = [
    "<s> a cat sat </s>", # <s>は文頭、</s>は文末トークン
    "<s> a dog sat </s>",
    "<s> the cat ran </s>",
    "<s> the dog ran </s>"
]

# トークン化
tokenized_corpus = [line.split() for line in test_corpus]

# 語彙の作成
all_words = [word for sentence in tokenized_corpus for word in sentence]
vocab = sorted(list(set(all_words)))
vocab_size = len(vocab)
word_to_index = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for i, word in enumerate(vocab)}

print("Vocabulary:", vocab)
print("Vocabulary Size:", vocab_size)

# Bigramカウント
bigram_counts = defaultdict(Counter)
unigram_counts = Counter()

for sentence in tokenized_corpus:
    for i in range(len(sentence) - 1):
        prev_word = sentence[i]
        curr_word = sentence[i+1]
        bigram_counts[prev_word][curr_word] += 1
        unigram_counts[prev_word] += 1

print("Bigram Counts (例: '<s>'): ', dict(bigram_counts['<s>'])")
print("Unigram Counts (例: '<s>'): ", unigram_counts['<s>'])

Vocabulary: ['</s>', '<s>', 'a', 'cat', 'dog', 'ran', 'sat', 'the']
Vocabulary Size: 8
Bigram Counts (例: '<s>'): ', dict(bigram_counts['<s>'])
Unigram Counts (例: '<s>'):  4


In [10]:
def get_laplace_bigram_prob(prev_word,
                            curr_word,
                            bigram_counts,
                            unigram_counts,
                            vocab_size,
                            k=1):
    ''' ラプラススムージング (Add-k) を用いたBigram確率の計算 '''
    numerator = bigram_counts[prev_word][curr_word] + k
    denominator = unigram_counts[prev_word] + k * vocab_size
    return numerator / denominator

# テスト
prob_cat_given_a = get_laplace_bigram_prob("a", "cat", bigram_counts, unigram_counts, vocab_size)
print(f"\nP(cat | a) (Laplace, Add-1): {prob_cat_given_a:.4f}") # (1+1)/(2+1*V)

prob_dog_given_the = get_laplace_bigram_prob("the", "dog", bigram_counts, unigram_counts, vocab_size)
print(f"P(dog | the) (Laplace, Add-1): {prob_dog_given_the:.4f}") # (1+1)/(2+1*V)

prob_sat_given_unknown = get_laplace_bigram_prob("unknown_word", "sat", bigram_counts, unigram_counts, vocab_size)
print(f"P(sat | unknown_word) (Laplace, Add-1): {prob_sat_given_unknown:.4f}") # 1/V


P(cat | a) (Laplace, Add-1): 0.2000
P(dog | the) (Laplace, Add-1): 0.2000
P(sat | unknown_word) (Laplace, Add-1): 0.1250


### 4.2 線形補間 (Linear Interpolation)

より高性能なスムージング手法の一つに線形補間があります。これは、異なる次数のn-gramモデルの予測を重み付きで組み合わせるものです。
例えば、Trigramモデルの場合、TrigramのMLE、BigramのMLE、UnigramのMLEを以下のように補間します。

$P_{interp}(w_i | w_{i-2}, w_{i-1}) = \lambda_3 P_{ML}(w_i | w_{i-2}, w_{i-1}) + \lambda_2 P_{ML}(w_i | w_{i-1}) + \lambda_1 P_{ML}(w_i)$

ここで、$\lambda_1, \lambda_2, \lambda_3$ は重みで、$\lambda_1 + \lambda_2 + \lambda_3 = 1$ かつ $\lambda_j \ge 0$ です。
これらの重み $\lambda_j$ は、通常、開発セット（held-out data）を用いて、開発セットの尤度（またはパープレキシティ）を最大（最小）にするように学習されます。
Jelinek-Mercer Smoothingはこの線形補間の一種で、$\lambda$の値が文脈に依存する場合もあります。

Chen & Goodman (1996) の論文では、この線形補間をベースとした様々なスムージング手法が比較検討されています。

### 4.3 Katz Backoff と Kneser-Ney Smoothing (概念紹介)

*   **Katz Backoff:**
    高次のn-gram（例: Trigram）が訓練データで観測されなかった場合、その確率を推定するために、より低次のn-gram（例: Bigram）の情報に「バックオフ」します。その際、確率の総和が1になるように、割り引かれた確率質量を低次モデルに分配します。Good-Turing推定などのアイデアが使われます。

*   **Kneser-Ney Smoothing:**
    現在最も性能が良いとされるスムージング手法の一つです。Katz Backoffを改良したもので、特に低頻度のn-gramの確率推定において優れています。単語がどれだけ多様な文脈で出現するか（continuation probability）を考慮する点が特徴的です。

これらの高度なスムージング手法の実装は複雑なため、このノートブックでは扱いませんが、高性能な言語モデルを構築する上で重要であることを覚えておきましょう。

## 5. 言語モデルの評価指標

学習された言語モデルの性能を評価するための主要な指標は、**負対数尤度 (Negative Log-Likelihood, NLL)** と**パープレキシティ (Perplexity, PPL)** です。
これらの指標は、通常、学習に使われなかった**テストセット**に対して計算されます。

### 5.1 負対数尤度 (NLL)

テストセット $W_{test} = (w_1, w_2, \dots, w_N)$ （$N$はテストセットの総単語数）に対するモデルの負対数尤度は、

$NLL(W_{test}) = - \sum_{i=1}^{N} \log_2 P(w_i | \text{context}_i)$

ここで、$P(w_i | \text{context}_i)$ はモデルが予測する、文脈 $\text{context}_i$ の後に単語 $w_i$ が出現する確率です。
対数の底は2、e、10のいずれでも使われますが、2を使うと単位がビットになります。
**NLLは小さいほど、モデルがテストデータをより確からしいと評価していることを意味し、性能が良いとされます。**

### 5.2 パープレキシティ (Perplexity - PPL)

パープレキシティは、NLLをテストセットの総単語数 $N$ で割った平均負対数尤度（これはクロスエントロピー $H(p,q)$ の推定値に近い）を指数関数の肩に乗せたものです。

$PPL(W_{test}) = 2^{H(p,q)} = 2^{-\frac{1}{N} \sum_{i=1}^{N} \log_2 P(w_i | \text{context}_i)}$

**直感的な解釈:**
パープレキシティは、「言語モデルが次の単語を予測する際に、平均していくつの選択肢に均等に迷っているか」を示す指標と解釈できます。
例えば、PPLが100であれば、モデルは次の単語を予測する際に平均して100個の単語の中から1つを選ぶのと同じくらいの不確かさを持っている、と考えられます。
**PPLも小さいほど、モデルの性能が良いとされます。** 語彙サイズが小さいほど、PPLのベースラインも小さくなります。

### 5.3 NumPyによるPPLの計算実装

In [11]:
def calculate_perplexity(log_probabilities):
    '''
    一連の対数確率からパープレキシティを計算する関数
    Parameters:
        log_probabilities(list or np.array): 各予測ステップでの真の次の単語の対数確率
    Returns:
        float: パープレキシティ
    '''
    num_predictions = len(log_probabilities)
    if num_predictions == 0:
        return float('inf')
    
    # 平均負対数尤度
    # H = - (1/N) * Σ log(P(w_i | w_{i-1}))
    mean_log_prob = -np.mean(log_probabilities)
    perplexity = np.power(2, mean_log_prob)

    return perplexity

In [12]:
# テスト
# 例: 3つの単語を予測し、それぞれの真の単語の対数確率 (底2) が以下だったとする
log_probs_example = [math.log2(0.5), math.log2(0.25), math.log2(0.125)] # [-1, -2, -3]
# 平均NLL = -(-1 -2 -3)/3 = 6/3 = 2
# PPL = 2^2 = 4
ppl_example = calculate_perplexity(log_probs_example)
print(f"Log probabilities: {log_probs_example}")
print(f"Perplexity: {ppl_example:.4f}") # 期待値: 4.0

log_probs_perfect = [math.log2(1.0), math.log2(1.0)] # 常に確率1で予測
ppl_perfect = calculate_perplexity(log_probs_perfect)
print(f"\nLog probabilities (perfect): {log_probs_perfect}")
print(f"Perplexity (perfect): {ppl_perfect:.4f}") # 期待値: 1.0 (迷いがない)

log_probs_random_vocab100 = [math.log2(1/100)] * 10 # 語彙100でランダムに予測
ppl_random_vocab100 = calculate_perplexity(log_probs_random_vocab100)
print(f"\nLog probabilities (random guess, vocab=100): {log_probs_random_vocab100[0]:.2f} ...")
print(f"Perplexity (random guess, vocab=100): {ppl_random_vocab100:.4f}") # 期待値: 100.0

Log probabilities: [-1.0, -2.0, -3.0]
Perplexity: 4.0000

Log probabilities (perfect): [0.0, 0.0]
Perplexity (perfect): 1.0000

Log probabilities (random guess, vocab=100): -6.64 ...
Perplexity (random guess, vocab=100): 100.0000


## 6. 実験: 簡単なテキストコーパスでのn-gramモデル学習と評価

### 6.1 トイコーパスと前処理

In [13]:
# 簡単なテキスト前処理とトークン化関数（再掲）
import re
def preprocess_text_simple(text):
    '''テキストを単純に小文字化し、数字と記号を削除する'''
    text = text.lower()  # 小文字化
    text = re.sub(r'[^a-z0-9\s]', '', text)  # 英小文字，数字，スペース以外を削除
    text = re.sub(r'\s+', ' ', text).strip()  # 連続するスペースを1つにまとめ、前後のスペースを削除
    return text

def tokenize_simple(text):
    '''テキストを単語に分割する'''
    return text.split(' ')

In [14]:
# 簡単なトイコーパス
toy_corpus = [
    "the cat sat on the mat",
    "the dog sat on the log",
    "a cat chased a dog",
    "a dog chased the cat"
]

# 前処理とトークナイズ、文頭・文末記号の追加
BOS = "<s>" # Begin Of Sentence
EOS = "</s>" # End Of Sentence
tokenized_toy_corpus = []
for sentence in toy_corpus:
    processed = preprocess_text_simple(sentence)
    tokens = [BOS] + tokenize_simple(processed) + [EOS]
    tokenized_toy_corpus.append(tokens)

print("Tokenized Toy Corpus:")
for sent_toks in tokenized_toy_corpus:
    print(sent_toks)

# 語彙の再作成
all_toy_words = [word for sent_toks in tokenized_toy_corpus for word in sent_toks]
toy_vocabulary = sorted(list(set(all_toy_words)))
toy_vocab_size = len(toy_vocabulary)
toy_word_to_idx = {word: i for i, word in enumerate(toy_vocabulary)}

print("\nToy Vocabulary:", toy_vocabulary)
print("Toy Vocabulary Size:", toy_vocab_size)

Tokenized Toy Corpus:
['<s>', 'the', 'cat', 'sat', 'on', 'the', 'mat', '</s>']
['<s>', 'the', 'dog', 'sat', 'on', 'the', 'log', '</s>']
['<s>', 'a', 'cat', 'chased', 'a', 'dog', '</s>']
['<s>', 'a', 'dog', 'chased', 'the', 'cat', '</s>']

Toy Vocabulary: ['</s>', '<s>', 'a', 'cat', 'chased', 'dog', 'log', 'mat', 'on', 'sat', 'the']
Toy Vocabulary Size: 11


### 6.2 n-gramカウントと確率計算 (ラプラススムージング)

In [15]:
def count_ngrams(tokenized_corpus, n):
    """ n-gramとその頻度をカウントする """
    ngram_counts = defaultdict(Counter) # history -> current_word -> count
    history_counts = Counter()         # history -> count
    
    for sentence_tokens in tokenized_corpus:
        # n-1個のBOSを文頭に追加して履歴を作成しやすくする
        padded_sentence = [BOS]*(n-1) + sentence_tokens
        for i in range(n-1, len(padded_sentence)):
            history = tuple(padded_sentence[i-(n-1) : i]) # (n-1)長のタプル
            current_word = padded_sentence[i]
            
            ngram_counts[history][current_word] += 1
            history_counts[history] += 1
            
    return ngram_counts, history_counts

def get_ngram_laplace_prob(history_tuple, current_word, ngram_counts, history_counts, vocab_size, k=1):
    """ ラプラススムージングを用いたn-gram条件付き確率 """
    # history_tuple は (w_{i-n+1}, ..., w_{i-1})
    count_ngram = ngram_counts[history_tuple][current_word]
    count_history = history_counts[history_tuple]
    
    if count_history == 0 and k == 0: # Add-0で履歴がない場合は均等確率 (またはより低次のモデルへ)
        return 1.0 / vocab_size 
        
    prob = (count_ngram + k) / (count_history + k * vocab_size)
    return prob

# Bigramモデル (n=2)
bigram_lm_counts, bigram_history_counts = count_ngrams(tokenized_toy_corpus, n=2)

# Trigramモデル (n=3)
trigram_lm_counts, trigram_history_counts = count_ngrams(tokenized_toy_corpus, n=3)

# 例: P(cat | <s>) for Bigram
prob_cat_given_bos_bi = get_ngram_laplace_prob(tuple([BOS]), "cat", bigram_lm_counts, bigram_history_counts, toy_vocab_size)
print(f"P(cat | <s>) [Bigram, Laplace]: {prob_cat_given_bos_bi:.4f}")

# 例: P(sat | a, cat) for Trigram
prob_sat_given_a_cat_tri = get_ngram_laplace_prob(tuple([BOS, "a", "cat"][-2:]), "sat", # 履歴は直前2単語
                                                 trigram_lm_counts, trigram_history_counts, toy_vocab_size)
print(f"P(sat | a cat) [Trigram, Laplace]: {prob_sat_given_a_cat_tri:.4f}")

P(cat | <s>) [Bigram, Laplace]: 0.0526
P(sat | a cat) [Trigram, Laplace]: 0.0833


### 6.3 テストセットでのパープレキシティ計算

In [16]:
# テスト用文 (訓練データにはないが、語彙は共通と仮定)
test_sentence_tokens = [BOS, "the", "dog", "chased", "the", "mat", EOS]

def calculate_sentence_perplexity(sentence_tokens, n, ngram_counts, history_counts, vocab_size, smoothing_k=1):
    """ 1つの文に対するパープレキシティを計算 (n-gramモデル) """
    log_prob_sum = 0.0
    num_predicted_words = 0 # 実際に確率を計算した単語数 (BOSは除く)

    # n-1個のBOSでパディング
    padded_sentence = [BOS]*(n-1) + sentence_tokens
    
    for i in range(n-1, len(padded_sentence)):
        history = tuple(padded_sentence[i-(n-1) : i])
        current_word = padded_sentence[i]
        
        # BOSを予測対象にはしない (通常、文頭記号の確率は考慮しないか、別途扱う)
        # ただし、ここでは単純化のため、P(word1 | <s>) から計算開始
        # EOSの確率 P(</s> | context) は計算する
        
        prob = get_ngram_laplace_prob(history, current_word, ngram_counts, history_counts, vocab_size, k=smoothing_k)
        if prob == 0: # ゼロ確率が発生した場合 (スムージングが不十分など)
            # log(0) を避けるために非常に小さい値を代入するか、エラー処理
            # print(f"Warning: Zero probability for P({current_word} | {history})")
            log_prob_sum += math.log2(1e-100) # 非常に小さい確率の対数
        else:
            log_prob_sum += math.log2(prob)
        
        # 文末記号EOSは予測されるが、次の単語の履歴にはならない
        if current_word != BOS : # BOS自身を予測する確率は数えない
             num_predicted_words +=1
             
    if num_predicted_words == 0: return float('inf')
    
    average_neg_log_likelihood = -log_prob_sum / num_predicted_words
    perplexity = np.power(2, average_neg_log_likelihood)
    return perplexity

# BigramモデルでのPPL
ppl_bigram_test = calculate_sentence_perplexity(test_sentence_tokens, 2, 
                                                bigram_lm_counts, bigram_history_counts, 
                                                toy_vocab_size, smoothing_k=1)
print(f"Test sentence PPL (Bigram, Laplace Add-1): {ppl_bigram_test:.2f}")

# TrigramモデルでのPPL
ppl_trigram_test = calculate_sentence_perplexity(test_sentence_tokens, 3,
                                                 trigram_lm_counts, trigram_history_counts,
                                                 toy_vocab_size, smoothing_k=1)
print(f"Test sentence PPL (Trigram, Laplace Add-1): {ppl_trigram_test:.2f}")

# よりスムージングを弱く (Add-0.01)
ppl_bigram_add_k = calculate_sentence_perplexity(test_sentence_tokens, 2, 
                                                 bigram_lm_counts, bigram_history_counts, 
                                                 toy_vocab_size, smoothing_k=0.01)
print(f"Test sentence PPL (Bigram, Laplace Add-0.01): {ppl_bigram_add_k:.2f}")

Test sentence PPL (Bigram, Laplace Add-1): 8.66
Test sentence PPL (Trigram, Laplace Add-1): 9.66
Test sentence PPL (Bigram, Laplace Add-0.01): 3.37


## 7. 考察

*   **n-gram言語モデルの基本:**
    *   マルコフ仮定に基づいて、直前の数単語の履歴から次の単語の出現確率をモデル化する単純かつ効果的な手法です。
    *   $n$ の値を大きくするほど長い文脈を考慮できますが、データスパースネス問題が深刻になります。一般的にTrigram ($n=3$) や4-gram程度が実用的な上限とされることが多いです。
*   **スムージングの重要性:**
    *   訓練データに出現しなかったn-gramに対してゼロでない確率を割り当てるために不可欠です。ラプラススムージングは最も単純ですが、より高度な手法（Kneser-Neyなど）が一般的には高性能です。
    *   スムージングの度合い（例: Add-kの $k$ の値）は、モデルの性能に影響を与えます。
*   **パープレキシティ (PPL):**
    *   言語モデルの性能を測る標準的な指標であり、モデルがテストデータに対してどれだけ「驚いているか」を示します。低いほど良いモデルです。
    *   異なるモデルや異なるコーパスでPPLを比較する際は、語彙サイズや前処理が同じであることを確認する必要があります。
*   **n-gramモデルの限界:**
    *   **長期依存性の欠如:** n-gramの「n」で定義される固定長の文脈しか考慮できないため、それより遠く離れた単語間の依存関係を捉えることができません。
    *   **意味的類似性の欠如:** "cat" と "dog" のように意味が似ていても、異なる単語として扱われるため、これらの単語が同じような文脈で出現するという知識を利用できません。
    *   **データスパースネス:** $n$ が大きくなると、観測されるn-gramの数が急増し、ほとんどのn-gramの出現頻度が0か非常に小さくなります。

これらの限界を克服するために、ニューラルネットワークを用いた言語モデル（例: RNN LM、Transformer LM）や、単語の意味を密なベクトルで表現する単語埋め込み（Word Embeddings）といった技術が登場します。